# ChatGoogleGenerativeAI

Access Google's `gemini` and `gemini-vision` models, as well as other generative models through `ChatGoogleGenerativeAI` class in the `langchain-google-genai` integration.

In [ ]:
%pip install -U langchain-google-genai

## Example usage

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

In a realm where words unfurl,
Where thoughts converge and fancies swirl,
There stands a tower, grand and tall,
LangChain, its name, a beacon's call.

Within its chambers, vast and deep,
A symphony of voices leap,
From every corner, near and far,
The echoes of humanity's star.

With keys of code, they weave their spells,
Crafting tales that truth excels,
A tapestry of language, rich and bold,
Where stories bloom and dreams unfold.

From realms of science, deep and vast,
To whispers of love that forever last,
LangChain embraces every hue,
A kaleidoscope of all things true.

Its algorithms, with grace untold,
Unravel mysteries, yet untold,
Connecting minds, bridging the gap,
In a digital dance, they never nap.

Among its ranks, a noble band,
Developers, united, hand in hand,
With passion fierce, they toil and strive,
To keep LangChain's flame alive.

They nurture it with tender care,
Nurturing its growth, beyond compare,
A testament to human might,
A beacon of progress, shining bright.



## Streaming and Batching

`ChatGoogleGenerativeAI` natively supports streaming and batching. Below is an example.

In [4]:
for chunk in llm.stream("Write a limerick about LLMs."):
    print(chunk.content)
    print("---")
# Note that each chunk may contain more than one "token"

Sure, here is a limerick about LLMs:

There once was
---
 an LLM named Burt,
Whose language skills made him an expert.
He could write code,
Compose poetry, and more,
And he even
---
 wrote this limerick for sport.
---


In [7]:
results = llm.batch(
    [
        "What's 2+2?",
        "What's 3+5?",
    ]
)
for res in results:
    print(res.content)

4
8


## Multimodal support

To provide an image, pass a human message with contents of type `List[dict]`, where each dict contains either an image value (type of `image_url`) or a text (type of `text`) value.
The value of `image_url` can be any of the following:

- A public image URL
- An accessible gcs file (e.g., "gcs://path/to/file.png")
- A local file path
- A base64 encoded image (e.g., `data:image/png;base64,abcd124`)
- A PIL image


In [14]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro-vision")
# example
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "What's in this image?",
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": "https://picsum.photos/seed/picsum/200/300"},
    ]
)
llm.invoke([message])

AIMessage(content=' A snow-capped mountain at sunset.')

## Gemini Prompting FAQs

As of the time this doc was written (2024/12/12), Gemini has some restrictions on the types and structure of prompts it accepts. Specifically:
1. When providing multimodal (image) inputs, you are restricted to at most 1 message of "human" (user) type. You cannot pass multiple messages (though the single human message may have multiple content entries)
2. System messages are not accepted.
3. For regular chat conversations, messages must follow the human/ai/human/ai alternating pattern. You may not provide 2 AI or human messages in sequence.
4. Message may be blocked if they violate the safety checks of the LLM. In this case, the model will return an empty response.